In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
capture_time = 10.0

def load_timing(csv_file):
    df = pd.read_csv(csv_file)
    # I think the max size datagram is the status message.
    max_size_only = df["data.len"] == np.max(df["data.len"])
    ts = np.asarray(df[max_size_only]["frame.time_relative"])
    ts = ts[ts <= capture_time]
    dts = np.diff(ts)
    return ts, dts

In [ ]:
base = "/tmp/franka_timing"
subdirs = ["localhost", "soft", "realtime"]
modes = ["robot_fake", "robot_real"]

In [ ]:
timings = {}
max_dt = 0.0
# Figure out max dt.
for s, subdir in enumerate(subdirs):
    for m, mode in enumerate(modes):
        csv_file = f"{base}/{subdir}/{mode}.csv"
        ts, dts = load_timing(csv_file)
        max_dt = max(max_dt, dts.max())
        timings[s, m] = (ts, dts)

In [ ]:
_, axs = plt.subplots(
    nrows=len(subdirs), ncols=len(modes), figsize=(15, 15), squeeze=False,
)
for s, subdir in enumerate(subdirs):
    for m, mode in enumerate(modes):
        ts, dts = timings[s, m]
        plt.sca(axs[s, m])
        plt.plot(ts[1:], dts)
        plt.ylabel("dt (sec)")
        plt.xlabel("time (sec)")
        plt.xlim(0, capture_time)
        plt.ylim(0, max_dt)
        plt.title(f"{subdir}/{mode}")
plt.tight_layout()
plt.savefig("/tmp/timing_combined.png")